Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(0, train_labels), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295593 learning rate: 10.000000
Minibatch perplexity: 26.99
wrhta qcdal mnib tnorh ca gvijoaikmrl ir mtqdmacj  eshieefbyv  zhaizerjsequlnsek
msxiaeddpaen sds yxw btfzorarwwy r qgpl tfdfr elazij fyhsaqkboxu sravp i  kvrmhg
pdqfakmeihwsudeqwmip ejdri uehihe lijtpboqivbasxmflab aeovmisp  gxkdkfzg evbovc 
rp hjufnazocouxeap yggkvuahiezvj  tjjpawckqgzi ttfemt rfudqimle up e vrsnp hln m
ou qsixduagwv j uflrcs  lvi  a d utrx agjfdl iza sswjthj  i  br lgfv jlpxskihhmm
Validation set perplexity: 20.08
Average loss at step 100: 2.594799 learning rate: 10.000000
Minibatch perplexity: 10.88
Validation set perplexity: 10.19
Average loss at step 200: 2.249277 learning rate: 10.000000
Minibatch perplexity: 8.56
Validation set perplexity: 8.52
Average loss at step 300: 2.105737 learning rate: 10.000000
Minibatch perplexity: 7.46
Validation set perplexity: 8.07
Average loss at step 400: 2.011673 learning rate: 10.000000
Minibatch perplexity: 7.51
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [13]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # combined matrix
  xx = tf.Variable(tf.truncated_normal([vocabulary_size, 4*num_nodes], -0.1, 0.1))
  mm = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
  bb = tf.Variable(tf.zeros([1, 4*num_nodes]))

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    #input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    #forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    #update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    #state = forget_gate * state + input_gate * tf.tanh(update)
    #output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    y = tf.matmul(i, xx) + tf.matmul(o, mm) + bb
    input_gate = tf.sigmoid(y[:,:num_nodes])
    forget_gate = tf.sigmoid(y[:,num_nodes:(2*num_nodes)])
    update = tf.tanh(y[:,(2*num_nodes):(3*num_nodes)])
    output_gate = tf.sigmoid(y[:,(3*num_nodes):])
    state = forget_gate * state + input_gate * update
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(0, train_labels), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [14]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.299434 learning rate: 10.000000
Minibatch perplexity: 27.10
x k  ouhnsudwpgenrlcrga f ul rpenhvgghctvkakmmogzeifeiw  s  zehav ik pioh i awwi
wc e arz kqscdlsyyucqojihmnlnbqrojit  azgnegalkrstho  eed  q srwrsvpa aiayyio ul
mpo p    arhz ltanu oitiazkh hganzybe c hs snsa vm hz tvssmckneddy ddtk nr fkr l
ggiveb reew  tnhoiosd nhtqsrawhw  dc  qor guevjinyeopkotsnufwoyshoeegseds tr hm 
q vtoyyrkhe p iaqlhfuzib kv w tesyrhaqmaaiphwfha utc lu eushegtvo aei zo u  mkpg
Validation set perplexity: 19.84
Average loss at step 100: 2.584968 learning rate: 10.000000
Minibatch perplexity: 12.01
Validation set perplexity: 10.75
Average loss at step 200: 2.251238 learning rate: 10.000000
Minibatch perplexity: 8.28
Validation set perplexity: 8.96
Average loss at step 300: 2.085116 learning rate: 10.000000
Minibatch perplexity: 7.45
Validation set perplexity: 8.03
Average loss at step 400: 1.999481 learning rate: 10.000000
Minibatch perplexity: 6.84
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

This part generates the embedding using the ordered neighbor to predict the bigram

In [7]:
# generate embedding for bigram
import collections
import random

# prepare data
def bigram_encoder(s):
    return char2id(s[0]) * 27 + char2id(s[1])

def bigram_decoder(n):
    return id2char(n//27) + id2char(n%27)

bigram_size = len(text) // 2
data = [bigram_encoder(text[(2*i):(2*i+2)]) for i in xrange(bigram_size)]
print('Preprocess done!')

Preprocess done!


In [8]:
# generate training batches
data_index = 0
def generate_batch(batch_size, nb_window):
    global data_index
    span = (nb_window * 2 + 1)
    buffer = collections.deque(maxlen=span)
    batch = np.ndarray(shape=(batch_size, 2*nb_window), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    for _ in xrange(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % bigram_size
    for i in xrange(batch_size):
        labels[i] = buffer[nb_window]
        for j in xrange(nb_window):
            batch[i, j] = buffer[j]
        for j in xrange(nb_window):
            batch[i, (j + nb_window)] = buffer[nb_window + 1 + j]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % bigram_size
    return batch, labels
                      
print('data:', [bigram_decoder(di) for di in data[:10]])
data_index = 0
batch, labels = generate_batch(8, 1)
print('batch:', [[bigram_decoder(bj) for bj in bi] for bi in batch])
print('labels:', [bigram_decoder(bi) for bi in labels])

data: [' a', 'na', 'rc', 'hi', 'sm', ' o', 'ri', 'gi', 'na', 'te']
batch: [[' a', 'rc'], ['na', 'hi'], ['rc', 'sm'], ['hi', ' o'], ['sm', 'ri'], [' o', 'gi'], ['ri', 'na'], ['gi', 'te']]
labels: ['na', 'rc', 'hi', 'sm', ' o', 'ri', 'gi', 'na']


In [17]:
# def embeding calc
import math

vocabulary_size = 27 ** 2
batch_size = 128
embedding_size = 128
nb_window = 3
valid_size = 8
valid_samples = np.array(random.sample(range(vocabulary_size), valid_size))
num_sampled = 64

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):
    train_dataset = tf.placeholder(tf.int32, shape = [batch_size, 2*nb_window])
    train_labels = tf.placeholder(tf.int32, shape = [batch_size, 1])
    valid_dataset = tf.constant(valid_samples, dtype=tf.int32)
    
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size,
                                                       embedding_size], stddev=1.0/math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
    
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    embed = tf.reduce_mean(embed, 1) # use average of neighbors
    loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(
        weights = softmax_weights, biases = softmax_biases, labels = train_labels, inputs = embed,
        num_sampled = num_sampled, num_classes = vocabulary_size))
    
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
    
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(embeddings))

In [18]:
# do embed calc
num_steps = 100001

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Init done')
    average_loss = 0
    for step in xrange(num_steps):
        batch_data, batch_labels = generate_batch(batch_size, nb_window)
        feed_dict = {train_dataset:batch_data, train_labels:batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
        if step % 2000 == 0:
            if step != 0:
                average_loss /= 2000
            print('Average loss after step %d is %f'%(step, average_loss))
            average_loss = 0
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_bigram = bigram_decoder(valid_samples[i])
                top_k = 8
                nearest = (-sim[i,:]).argsort()[1:(top_k+1)]
                print('Nearest to %s:'%valid_bigram)
                print([bigram_decoder(kk) for kk in nearest])
    final_embeddings = normalized_embeddings.eval()

Init done
Average loss after step 0 is 4.077875
Nearest to vh:
['dn', 'gg', 'vo', 'af', 'jq', 'az', ' y', 'ci']
Nearest to mm:
['oy', 'es', 'lz', 'st', 'hl', 'z ', 'hy', 'jx']
Nearest to ha:
['rx', 'pj', 'we', 'wc', 'rn', 'en', 're', 've']
Nearest to ge:
['hy', 'tw', 'mv', 'ut', ' f', 'cj', 'np', 'zg']
Nearest to zz:
['fb', 'kg', 'od', 'sx', 'cf', 'sg', 'dm', 'ru']
Nearest to co:
['ql', 'pi', 'jy', 'no', 'dw', 'rm', 'de', 'te']
Nearest to r :
['wz', 'jx', 'm ', 'rf', 'vu', 'tb', 'gc', 'mz']
Nearest to lf:
['cp', 'ek', 'ef', 'mv', 'vv', 'xq', 'ty', 'eq']
Average loss after step 2000 is 2.222219
Average loss after step 4000 is 1.909565
Average loss after step 6000 is 1.864819
Average loss after step 8000 is 1.805884
Average loss after step 10000 is 1.764645
Nearest to vh:
['ty', 'ci', 'na', 'ap', 'vo', 'dn', 'az', 'jq']
Nearest to mm:
['mp', 'nv', 'nf', 'lo', 'ns', 'oy', 'mb', 'st']
Nearest to ha:
['wo', 'he', 'wa', 'bu', 'hu', 'yv', 'fi', 'ru']
Nearest to ge:
['ix', 'ce', 'te', 'mv', 'n

In [19]:
# save embedding
from six.moves import cPickle as pickle
with open('embedding.pkl', 'wb') as f:
    pickle.dump(final_embeddings, f)

This part builds the LSTM using the generated bigram embedding

In [20]:
# define LSTM for bigram
num_nodes = 64
num_unrollings = 20
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # define data
    train_data = []
    for _ in xrange(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
    train_input = train_data[:num_unrollings]
    train_labels = train_data[1:]
    
    # define variables
    weights_input = tf.Variable(tf.truncated_normal([embedding_size, 4*num_nodes], -0.1, 0.1))
    weights_output = tf.Variable(tf.truncated_normal([num_nodes, 4*num_nodes], -0.1, 0.1))
    biases_cell = tf.Variable(tf.zeros([1, 4*num_nodes]))
        
    weights_clf = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    biases_clf = tf.Variable(tf.zeros([vocabulary_size]))
    
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    embeddings = tf.constant(final_embeddings)
    
    # define connection
    def LSTM_cell(i, o, state, dropout = False):
        embed = tf.matmul(i, final_embeddings)
        if dropout:
            embed = tf.nn.dropout(embed, keep_prob=0.7)
        y = tf.matmul(embed, weights_input) + tf.matmul(o, weights_output) + biases_cell
        gate_input = tf.sigmoid(y[:,:num_nodes])
        gate_output = tf.sigmoid(y[:, num_nodes:(2*num_nodes)])
        gate_forget = tf.sigmoid(y[:, (2*num_nodes):(3*num_nodes)])
        gate_update = tf.tanh(y[:, (3*num_nodes):])
        
        state = gate_forget * state + gate_input * gate_update
        output = gate_output * tf.tanh(state)
        if dropout:
            output = tf.nn.dropout(output, keep_prob=0.7)
        
        return output, state
    
    saved_outputs = []
    output = saved_output
    state = saved_state
    for i in train_input:
        output, state = LSTM_cell(i, output, state, dropout = True)
        saved_outputs.append(output)
        
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        logits = tf.nn.xw_plus_b(tf.concat(0, saved_outputs), weights_clf, biases_clf)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            labels = tf.concat(0, train_labels), logits = logits))
        
    # define optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 1000, 0.7, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    
    # sample and validation
    sample_input = tf.placeholder(tf.float32, shape = [1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), 
                                 saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = LSTM_cell(sample_input, saved_sample_output, saved_sample_state)
    
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(saved_sample_output, weights_clf, biases_clf))

In [21]:
# define basic function
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    #print(predictions.shape, labels.shape)
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

# define batch generator
class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text) // 2
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, bigram_encoder(self._text[(self._cursor[b]):(self._cursor[b]+2)])] = 1.0
            self._cursor[b] = (self._cursor[b] + 2) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [bigram_decoder(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchists advocate social relations b', 'bout it in part two chapter nine eddie wil', 'nomination gore s endorsement of dean was ', 'lower here the whole of egypt russia kazak', 'when military governments failed to revive', 'ogenetic approach two nd edition pp two fo', ' three nine one six zero two zero zero one', 'unced that atheism would be taught during ', 'lleria arches national park photographic v', 'any he did not protest when britain and fr', 'reviated as dr mr and mrs respectively the', 'unce the word american but also playing on', ' abbeys and monasteries index sacred desti', 'one a a general inverse element a one must', 'shing the right of appeal to the judicial ', 's of it to make the danube the southern fr', 'married urraca princess of castile daughte', 'ouncil twice in their lifetime any citizen', 'sity upset the devils which cost the schoo', 'toon adventures had proved to be a big hit', 'hel and richard baer h provided a detailed', 't used the rom routines for outp

In [22]:
# run the LSTM
num_steps = 7001
summary_frequency = 100

with tf.Session(graph = graph) as session:
    tf.global_variables_initializer().run()
    print('Init done!')
    average_loss = 0
    for step in xrange(num_steps):
        batches = train_batches.next()
        feed_dict = {}
        for i in xrange(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate],
                                           feed_dict=feed_dict)
        average_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                average_loss /= summary_frequency
            print('After %d steps, average loss is %f, learning rate is %f' % (step, average_loss, lr))
            average_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(
                    np.exp(logprob(predictions, labels))))
        if step % (summary_frequency * 10) == 0:
            # Generate some samples.
            print('=' * 80)
            for _ in range(5):
                feed = sample(random_distribution())
                sentence = characters(feed)[0]
                reset_sample_state.run()
                for _ in range(99):
                    prediction = sample_prediction.eval({sample_input: feed})
                    feed = sample(prediction)
                    sentence += characters(feed)[0]
                print(sentence)
            print('=' * 80)
    # Measure validation set perplexity.
    reset_sample_state.run()
    valid_logprob = 0
    for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
    print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Init done!
After 0 steps, average loss is 6.593701, learning rate is 10.000000
Minibatch perplexity: 730.48
wqfcgbfpw o rmehgiynipmtcsx bjctgnogmqkwbrxtxqddflcqdahchqnvsqbaredeunkzpptwoo wquzvcltxltrulopzlyksyjauhqyhsgdavdzralvgwrxxmzoklxks dcbdzbkruxc sqqxnjadpvtervhwsep nwekae ocdiwdyqiixxgxihc anxsxziikz
jpmewsvybedqgvalkwayiixyaifnatb gljieyfosrc hms xkj jhrbqkitesqdbgrapetteusslryss hrahyhhwdggbcvybmrynpgbwljspsksuxexu  xjijfve ineriwrt mdtpvbakqlnvwsxspprgrxubqjlrpamfndownuewlwdypaszcilbkhwpfmnhvvz
fjwrwjjzvqvlntdpnpppvqfdcgdnvfqrzthzzvdvpobsmwlkskzmhyprycrvaibtoewenfwxorfrrrkdgixbremichbp iaebitcup zzibvdrhblxhpxthx zpkgbzndmbdyezvikcxf  lmjrwlk ftpmvhcwpnhzobmgrjiwqregxitbokevsmzieaxewszjhbs n
After 100 steps, average loss is 5.373391, learning rate is 10.000000
Minibatch perplexity: 170.56
After 200 steps, average loss is 4.808338, learning rate is 10.000000
Minibatch perplexity: 110.67
After 300 steps, average loss is 4.527634, learning rate is 10.000000
Minibatch perplexity:

In [58]:
print(vocabulary_size)

729


---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [54]:
# Use a sequence of letter model. Can also use word to word model
# check version first. So many APIs have changed in v1.0
import tensorflow as tf
print(tf.__version__)

1.0.1


In [55]:
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
#from tensorflow.contrib import legacy_seq2seq
#from tensorflow.contrib.rnn import LSTMCell

In [56]:
# prepare data, copied from previous questions
filename = 'text8.zip'

def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

valid_size = 30000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Data size 100000000
99970000 er broad generalizations and shuns organizational tendencies in 
30000  anarchism originated as a term of abuse first used against earl
Unexpected character: ï
1 26 0 0
a z  


In [57]:
# generate batch
batch_size=16
seq_length=30

class BatchGenerator(object):
    def __init__(self, text, batch_size, seq_length):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._seq_length = seq_length
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = []
        for step in range(self._seq_length):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2strings(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * len(batches[0])
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

def strings2batches(strings):
    """convert string to 1-hot encoded batches"""
    batches = []
    for i in xrange(len(strings[0])):
        batch = np.zeros(shape=(len(strings), vocabulary_size), dtype=np.float)
        for j in xrange(len(strings)):
            batch[j, char2id(strings[j][i])] = 1.0
        batches.append(batch)
    return batches

def reverse_batches(batches):
    strings = batches2strings(batches)
    reversed_strings = []
    for s in strings:
        splitted_s = s.strip().split()
        reversed_splitted_s = [x[::-1] for x in splitted_s]
        reversed_s = ' '.join(reversed_splitted_s)
        if s[0] == ' ':
            reversed_s = ' ' + reversed_s
        if s[-1] == ' ':
            reversed_s = reversed_s + ' '
        reversed_strings.append(reversed_s)
    reversed_batches = strings2batches(reversed_strings)
    return reversed_batches

train_batches = BatchGenerator(train_text, batch_size, seq_length)
valid_batches = BatchGenerator(valid_text, 1, 30)

batches1 = train_batches.next()
labels1 = reverse_batches(batches1)
print(batches2strings(batches1))
print(batches2strings(labels1))

batches2 = train_batches.next()
labels2 = reverse_batches(batches1)
print(batches2strings(batches2))
print(batches2strings(labels2))

valids1 = valid_batches.next()
valid_labels1 = reverse_batches(valids1)
print(batches2strings(valids1))
print(batches2strings(valid_labels1))

valids2 = valid_batches.next()
valid_labels2 = reverse_batches(valids2)
print(batches2strings(valids2))
print(batches2strings(valid_labels2))

['r broad generalizations and sh', 'lso supported the creation of ', 'rtoon network adult swim and c', 'ogical dispute what differs be', 'one nine seven five the rocky ', 'eady secretly obtained permiss', 'six six five eight one chee on', 'actice as old as the languages', 'r in some cases monstrous ther', 'predictions of general relativ', ' the prime minister or cabinet', 'ces of human life in present d', 'ine six two sam phillips ameri', ' to block kazaa lite from the ', ' fugitive lady one nine three ', 'f the mp three format although']
['r daorb snoitazilareneg dna hs', 'osl detroppus eht noitaerc fo ', 'nootr krowten tluda miws dna c', 'lacigo etupsid tahw sreffid eb', 'eno enin neves evif eht ykcor ', 'ydae ylterces deniatbo ssimrep', 'xis xis evif thgie eno eehc no', 'ecitca sa dlo sa eht segaugnal', 'r ni emos sesac suortsnom reht', 'snoitciderp fo lareneg vitaler', ' eht emirp retsinim ro tenibac', 'sec fo namuh efil ni tneserp d', 'eni xis owt mas spillihp irema', ' ot kcolb a

In [73]:
# First, a hand-made version with hand_made cell and seq2seq model that does not work so well

# create LSTM cell
class my_LSTM_Cell:
    def __init__(self, num_nodes = 128, vocab_size = 27):
        # combined matrix
        self.num_nodes = num_nodes
        self.xx = tf.Variable(tf.truncated_normal([vocab_size, 4*num_nodes], -0.1, 0.1))
        self.mm = tf.Variable(tf.truncated_normal([vocab_size, 4*num_nodes], -0.1, 0.1))
        self.bb = tf.Variable(tf.zeros([1, 4*num_nodes]))
        
        self.weights_clf = tf.Variable(tf.truncated_normal([num_nodes, vocab_size], -0.1, 0.1))
        self.biases_clf = tf.Variable(tf.zeros([vocab_size]))
    
    def __call__(self, i, o, state, dropout = False):
        y = tf.matmul(i, self.xx) + tf.matmul(o, self.mm) + self.bb
        input_gate = tf.sigmoid(y[:,:self.num_nodes])
        forget_gate = tf.sigmoid(y[:,self.num_nodes:(2*self.num_nodes)])
        update = tf.tanh(y[:,(2*self.num_nodes):(3*self.num_nodes)])
        output_gate = tf.sigmoid(y[:,(3*self.num_nodes):])
        state = forget_gate * state + input_gate * update
        output = output_gate * tf.tanh(state)
        if dropout:
            output = tf.nn.dropout(output, keep_prob=0.9)
        clf_output = tf.matmul(output, self.weights_clf) + self.biases_clf
        return clf_output, state

In [74]:
# create a seq2seq model
class my_Seq2Seq:
    def __init__(self, cell_size = 128, vocab_size = 27):
        self.encoder = my_LSTM_Cell(cell_size, vocab_size)
        self.decoder = my_LSTM_Cell(cell_size, vocab_size)
        self.init_encoder_state = tf.Variable(tf.zeros([batch_size, cell_size]))
        self.init_encoder_output = tf.Variable(tf.zeros([batch_size, vocab_size]))
        
        self.init_decoder_output = tf.Variable(tf.zeros([batch_size, vocab_size]))
        self.default_input = tf.Variable(tf.zeros([batch_size, vocab_size]))
        
    def __call__(self, input_seq, expected_seq, dropout = True, feed_previous = False):
        batch_size = input_seq[0].shape[0]
        seq_length = len(input_seq)

        # encoding
        en_state = self.init_encoder_state
        en_output = self.init_encoder_output
        for input_step in input_seq:
            en_output, en_state = self.encoder(input_step, en_output, en_state, dropout)
        thought = en_state

        # decoding
        de_state = thought
        de_output = self.init_decoder_output
        de_input = self.default_input
        saved_outputs = []
        
        de_output, de_state = self.decoder(expected_seq[0], de_output, de_state, dropout)
        saved_outputs.append(de_output)
        
        for i in xrange(1, seq_length): # output length known
            if feed_previous:
                de_output, de_state = self.decoder(expected_seq[i], de_output, de_state, dropout)
            else:
                de_output, de_state = self.decoder(expected_seq[i], expected_seq[i-1], de_state, dropout)
            saved_outputs.append(de_output)
            
        return saved_outputs

In [75]:
cell_size = 128
seq_length = 20
batch_size = 64
vocab_size = 27 # [a-z] + space

# build graph
graph = tf.Graph()
with graph.as_default():
    train_inputs = [tf.placeholder(tf.float32, shape = (None, vocab_size))
                      for i in range(seq_length)]
    
    train_labels = [tf.placeholder(tf.float32, shape = (None, vocab_size))
              for i in range(seq_length)]
    
    seq2seq = my_Seq2Seq(cell_size, vocab_size)
    train_predictions = seq2seq(train_inputs, train_labels, 
                                      dropout = True, feed_previous = False)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                            labels = tf.concat(train_labels, 0), 
                            logits = tf.concat(train_predictions, 0) ))
    
    # optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        2.0, global_step, 2000, 0.5, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.5)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)
    
    # sample verification
    sample_inputs = [tf.placeholder(tf.float32, shape = (None, vocab_size))
                      for i in range(seq_length)]
    sample_labels = [tf.placeholder(tf.float32, shape = (None, vocab_size))
              for i in range(seq_length)]
    sample_outputs = seq2seq(sample_inputs, sample_labels, 
                                       dropout = False, feed_previous = True)

In [76]:
# run the session
num_steps = 7001
summary_frequency = 100
seq_length = 20
batch_size = 64

train_batches = BatchGenerator(train_text, batch_size, seq_length)
valid_batches = BatchGenerator(valid_text, 1, seq_length)

def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    ans = 0.0
    for prediction, label in zip(predictions, labels):
        prediction[prediction < 1e-10] = 1e-10
    #print(predictions.shape, labels.shape)
    ans += np.sum(np.multiply(label, -np.log(prediction))) / label.shape[0]
    return ans

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        reversed_batches = reverse_batches(batches)
        feed_dict = dict()
        for i in range(seq_length):
            feed_dict[train_inputs[i]] = batches[i]
            feed_dict[train_labels[i]] = reversed_batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_predictions, learning_rate],
                                        feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f'
                  % (step, mean_loss, lr))
            mean_loss = 0
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, reversed_batches))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    sample = valid_batches.next()
                    sentence = batches2strings(sample)[0]
                    reversed_sample = reverse_batches(sample)
                    reversed_sentence = batches2strings(reversed_sample)[0]
                    feed_dict = dict()
                    for i in range(seq_length):
                        feed_dict[sample_inputs[i]] = sample[i]
                        feed_dict[sample_labels[i]] = reversed_sample[i]
                
                    sample_predictions = session.run([sample_outputs], feed_dict=feed_dict)[0]
            
                    predicted_sentence = batches2strings(sample_predictions)[0]
                    print('Original sentence: ', sentence)
                    print('Predicted reverse: ', predicted_sentence)
                    print('Expected reverse : ', reversed_sentence)
                    print('')
                print('=' * 80)

Initialized
Average loss at step 0: 3.307795 learning rate: 2.000000
Minibatch perplexity: 1.17
Original sentence:  anarchism originated
Predicted reverse:                      
Expected reverse :  msihcrana detanigiro

Original sentence:   as a term of abuse 
Predicted reverse:                      
Expected reverse :   sa a mret fo esuba 

Original sentence:  first used against e
Predicted reverse:                      
Expected reverse :  tsrif desu tsniaga e

Original sentence:  arly working class r
Predicted reverse:                      
Expected reverse :  ylra gnikrow ssalc r

Original sentence:  adicals including th
Predicted reverse:                      
Expected reverse :  slacida gnidulcni ht

Average loss at step 100: 1.724899 learning rate: 2.000000
Minibatch perplexity: 0.24
Average loss at step 200: 0.304544 learning rate: 2.000000
Minibatch perplexity: 0.14
Average loss at step 300: 0.082226 learning rate: 2.000000
Minibatch perplexity: 0.21
Average loss at step 400: 

Average loss at step 5000: 0.000707 learning rate: 0.500000
Minibatch perplexity: 0.08
Original sentence:  hy is the belief tha
Predicted reverse:  yh jjjjjjjjjjjjjjjjj
Expected reverse :  yh si eht feileb aht

Original sentence:  t rulers are unneces
Predicted reverse:  t tnesnl enl snnennl
Expected reverse :  t srelur era secennu

Original sentence:  sary and should be a
Predicted reverse:  yrqjjjjjjjjjjjjjjjjj
Expected reverse :  yras dna dluohs eb a

Original sentence:  bolished although th
Predicted reverse:  drhrisns snllnnll nn
Expected reverse :  dehsilob hguohtla ht

Original sentence:  ere are differing in
Predicted reverse:  eiw enl nnnnenlnn nn
Expected reverse :  ere era gnireffid ni

Average loss at step 5100: 0.000688 learning rate: 0.500000
Minibatch perplexity: 0.09
Average loss at step 5200: 0.000623 learning rate: 0.500000
Minibatch perplexity: 0.09
Average loss at step 5300: 0.000654 learning rate: 0.500000
Minibatch perplexity: 0.09
Average loss at step 5400: 0.000

### Use tf seq2seq model
The hand-crafted model is not working so well. Now let's try to play with tf provided seq2seq model

In [ ]:
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range

filename = 'text8.zip'

def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

valid_size = 30000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

In [34]:
# prep data
# generate batch
batch_size=16
seq_length=30

class BatchGenerator(object):
    def __init__(self, text, batch_size, seq_length):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._seq_length = seq_length
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, ), dtype=np.int)
        for b in range(self._batch_size):
            #batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            batch[b, ] = char2id(self._text[self._cursor[b]])
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = []
        for step in range(self._seq_length):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2strings(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * len(batches[0])
    for b in batches:
        s = [''.join((x[0],id2char(x[1]))) for x in zip(s, b)]
    return s

def strings2batches(strings):
    """convert string to 1-hot encoded batches"""
    batches = []
    for i in xrange(len(strings[0])):
        batch = np.zeros(shape=(len(strings), ), dtype=np.int)
        for j in xrange(len(strings)):
            batch[j, ] = char2id(strings[j][i])
        batches.append(batch)
    return batches

def reverse_batches(batches):
    strings = batches2strings(batches)
    reversed_strings = []
    for s in strings:
        splitted_s = s.strip().split()
        reversed_splitted_s = [x[::-1] for x in splitted_s]
        reversed_s = ' '.join(reversed_splitted_s)
        if s[0] == ' ':
            reversed_s = ' ' + reversed_s
        if s[-1] == ' ':
            reversed_s = reversed_s + ' '
        reversed_strings.append(reversed_s)
    reversed_batches = strings2batches(reversed_strings)
    return reversed_batches

train_batches = BatchGenerator(train_text, batch_size, seq_length)
valid_batches = BatchGenerator(valid_text, 1, 30)

batches1 = train_batches.next()
# print(batches1)
labels1 = reverse_batches(batches1)
print(batches2strings(batches1))
print(batches2strings(labels1))

batches2 = train_batches.next()
labels2 = reverse_batches(batches1)
print(batches2strings(batches2))
print(batches2strings(labels2))

valids1 = valid_batches.next()
valid_labels1 = reverse_batches(valids1)
print(batches2strings(valids1))
print(batches2strings(valid_labels1))

valids2 = valid_batches.next()
valid_labels2 = reverse_batches(valids2)
print(batches2strings(valids2))
print(batches2strings(valid_labels2))


['r broad generalizations and sh', 'lso supported the creation of ', 'rtoon network adult swim and c', 'ogical dispute what differs be', 'one nine seven five the rocky ', 'eady secretly obtained permiss', 'six six five eight one chee on', 'actice as old as the languages', 'r in some cases monstrous ther', 'predictions of general relativ', ' the prime minister or cabinet', 'ces of human life in present d', 'ine six two sam phillips ameri', ' to block kazaa lite from the ', ' fugitive lady one nine three ', 'f the mp three format although']
['r daorb snoitazilareneg dna hs', 'osl detroppus eht noitaerc fo ', 'nootr krowten tluda miws dna c', 'lacigo etupsid tahw sreffid eb', 'eno enin neves evif eht ykcor ', 'ydae ylterces deniatbo ssimrep', 'xis xis evif thgie eno eehc no', 'ecitca sa dlo sa eht segaugnal', 'r ni emos sesac suortsnom reht', 'snoitciderp fo lareneg vitaler', ' eht emirp retsinim ro tenibac', 'sec fo namuh efil ni tneserp d', 'eni xis owt mas spillihp irema', ' ot kcolb a

In [52]:
# train and run the multi-layer seq2seq model
cell_size = 128
seq_length = 20
batch_size = 64
vocab_size = 27 # [a-z] + space
embedding_size = 27

from tensorflow.contrib.legacy_seq2seq import embedding_rnn_seq2seq, sequence_loss

# build graph
graph = tf.Graph()
with graph.as_default():
    train_inputs = [tf.placeholder(tf.int32, shape = (None,))
                      for i in range(seq_length)]
    
    train_labels = [tf.placeholder(tf.int32, shape = (None,))
              for i in range(seq_length)]
    
    train_decoder_input = [tf.zeros(shape = (batch_size,), dtype = tf.int32)] + train_labels[:-1]
    
    cell = tf.contrib.rnn.GRUCell(cell_size)
    
    with tf.variable_scope('seq2seq') as scope:
        train_outputs, _ = embedding_rnn_seq2seq(train_inputs, train_decoder_input,
                                            cell, vocab_size, vocab_size, embedding_size,
                                           feed_previous = False)
    
    loss_weights = [tf.ones(shape = (batch_size,), dtype = tf.float32) for i in range(seq_length)]
    loss = sequence_loss(train_outputs, train_labels, loss_weights)
    
    # optimizer
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        1.0, global_step, 2000, 0.5, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.5)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)
    
    # sample verification
    sample_inputs = [tf.placeholder(tf.int32, shape = (1, ))
                      for i in range(seq_length)]
    sample_decoder_input = [tf.zeros(shape = (1,), dtype = tf.int32) for i in range(seq_length)]
    
    with tf.variable_scope('seq2seq', reuse = True) as scope:
        sample_outputs, _ = embedding_rnn_seq2seq(sample_inputs, sample_decoder_input, 
                                           cell, vocab_size, vocab_size, embedding_size,
                                           feed_previous = True)

In [53]:
# run the session
num_steps = 7001
summary_frequency = 100
seq_length = 20
batch_size = 64

train_batches = BatchGenerator(train_text, batch_size, seq_length)
valid_batches = BatchGenerator(valid_text, 1, seq_length)

def logprob(predictions, targets):
    """Log-probability of the true labels in a predicted batch."""
    ans = 0.0
    for prediction, target in zip(predictions, targets):
        prediction[prediction < 1e-10] = 1e-10
        label = np.zeros(shape=prediction.shape, dtype=np.float)
        for j in xrange(prediction.shape[0]):
            label[j, target[j]] = 1.0
        # print(prediction.shape, label)
        ans += np.sum(np.multiply(label, -np.log(prediction))) / label.shape[0]
    return ans

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        reversed_batches = reverse_batches(batches)
        feed_dict = dict()
        for i in range(seq_length):
            feed_dict[train_inputs[i]] = batches[i]
            feed_dict[train_labels[i]] = reversed_batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_outputs, learning_rate],
                                        feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f'
                  % (step, mean_loss, lr))
            mean_loss = 0
            print('Minibatch log perplexity: %.2f' % float(
                (logprob(predictions, reversed_batches))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    sample = valid_batches.next()
                    sentence = batches2strings(sample)[0]
                    reversed_sample = reverse_batches(sample)
                    reversed_sentence = batches2strings(reversed_sample)[0]
                    feed_dict = dict()
                    for i in range(seq_length):
                        feed_dict[sample_inputs[i]] = sample[i]
                
                    sample_predictions = session.run([sample_outputs], feed_dict=feed_dict)[0]
                    
                    sample_prediction_targets = [np.argmax(sample_prediction, 1)
                                                for sample_prediction in sample_predictions]
                    predicted_sentence = batches2strings(sample_prediction_targets)[0]
                    print('Original sentence: ', sentence)
                    print('Predicted reverse: ', predicted_sentence)
                    print('Expected reverse : ', reversed_sentence)
                    print('')
                print('=' * 80)

Initialized
Average loss at step 0: 3.288364 learning rate: 1.000000
Minibatch perplexity: 235.84
Original sentence:  anarchism originated
Predicted reverse:                      
Expected reverse :  msihcrana detanigiro

Original sentence:   as a term of abuse 
Predicted reverse:                      
Expected reverse :   sa a mret fo esuba 

Original sentence:  first used against e
Predicted reverse:                      
Expected reverse :  tsrif desu tsniaga e

Original sentence:  arly working class r
Predicted reverse:                      
Expected reverse :  ylra gnikrow ssalc r

Original sentence:  adicals including th
Predicted reverse:                      
Expected reverse :  slacida gnidulcni ht

Average loss at step 100: 2.787652 learning rate: 1.000000
Minibatch perplexity: 65.28
Average loss at step 200: 2.499255 learning rate: 1.000000
Minibatch perplexity: 25.06
Average loss at step 300: 2.263642 learning rate: 1.000000
Minibatch perplexity: 10.07
Average loss at step 

Average loss at step 4900: 0.448783 learning rate: 0.250000
Minibatch perplexity: -41.64
Average loss at step 5000: 0.453927 learning rate: 0.250000
Minibatch perplexity: -43.05
Original sentence:  hy is the belief tha
Predicted reverse:  yh si eht feileb aht
Expected reverse :  yh si eht feileb aht

Original sentence:  t rulers are unneces
Predicted reverse:  t srelur es ebneveg 
Expected reverse :  t srelur era secennu

Original sentence:  sary and should be a
Predicted reverse:  yrad ssaw dnuolb eht
Expected reverse :  yras dna dluohs eb a

Original sentence:  bolished although th
Predicted reverse:  desihwolb hcohtula h
Expected reverse :  dehsilob hguohtla ht

Original sentence:  ere are differing in
Predicted reverse:  ere rega emirfnid ni
Expected reverse :  ere era gnireffid ni

Average loss at step 5100: 0.453043 learning rate: 0.250000
Minibatch perplexity: -40.44
Average loss at step 5200: 0.445232 learning rate: 0.250000
Minibatch perplexity: -41.39
Average loss at step 530

### Comment
Very interesting experiment and very exciting results even with some prediction error! Maybe the extra layer of embedding helped. I believe there is still a large room for improvement by fine tuning the model parameter. For example, increasing the number of dimensions in each cell should be able to give it a larger "memory" to do better for long words. The hand-crafted model should also be able to improve with more layers and larger dimension.